<a href="https://colab.research.google.com/github/Yanh12/MA_automatische_Zusammenfassung/blob/master/abstraktive_Phase/Model3/P_G_data_preprocessing_model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/My Drive/MA_colab/PG_textRank')
default_path = '/content/drive/My Drive/MA_colab/PG_textRank/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
import os
import hashlib
import struct
import subprocess
import collections
import tensorflow as tf
from tensorflow.core.example import example_pb2
import nltk
import pandas as pd

In [ ]:
import pickle, re
import nltk
#tokenize the source text
nltk.download('punkt')
from nltk.tokenize import word_tokenize
with open('/content/drive/My Drive/MA_colab/textRank_texts.p', 'rb') as filehandle:
  #list of list, each inner list contains several sentences
  cores_ngram = pickle.load(filehandle)
cores_ngram_tokenized = [word_tokenize(' '.join(x)) for x in cores_ngram]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
for i in range (400,410):
    print(cores_ngram_tokenized[i])

['compared', 'with', 'the', 'baseline', 'which', 'only', 'uses', 'the', 'forward', 'language', 'model', ',', 'our', 'experimental', 'results', 'show', 'that', 'the', 'additional', 'backward', 'language', 'model', 'is', 'able', 'to', 'gain', 'about', '0.5', 'bleu', 'points', ',', 'while', 'the', 'mi', 'trigger', 'model', 'gains', 'about', '0.4', 'bleu', 'points', '.', 'therefore', 'our', 'mi', 'trigger', 'model', 'selects', '“', 'is', '”', 'rather', 'than', '“', 'was', '”', '.9', 'this', 'example', 'illustrates', 'that', 'the', 'mi', 'trigger', 'model', 'is', 'capable', 'of', 'selecting', 'correct', 'words', 'by', 'using', 'longdistance', 'trigger', 'pairs', '.', 'and', 'both', 'the', 'numerator', 'and', 'denominator', 'have', 'up', 'to', '2', 'terminal', 'symbols', '.', 'where', 'the', 'approximation', 'is', 'based', 'on', 'the', 'nth', 'order', 'markov', 'assumption', '.', 'table', '3', 'shows', 'an', 'example', 'from', 'our', 'test', 'set', '.', 'for', 'example', ',', 'a', 'verb', 'i

In [ ]:
 #sentence-wise segment the summary
from nltk.tokenize.punkt import PunktSentenceTokenizer
senttokenizer = PunktSentenceTokenizer()
with open ('/content/drive/My Drive/MA_colab/abs_strV2.p', 'rb') as filehandle:
  abs_selected = pickle.load(filehandle)
for ab in abs_selected:
    ab = ab.lower()
    senttokenizer.train(ab)
abs_selected = [senttokenizer.tokenize(x) for x in abs_selected]
from nltk.tokenize import RegexpTokenizer
regTokenizer =  RegexpTokenizer(pattern=r"\s|[\]", gaps=True)

abs_sent = []
for ab in abs_selected:
    ab_sent = []
    for s in ab:
        s_new = '<s> ' + s + ' </s> '
        ab_sent.append(s_new)
    abs_sent.append(' '.join(ab_sent))
abs_sent = [regTokenizer.tokenize(x) for x in abs_sent]


In [ ]:
for i in range(768,777):
    print(abs_sent[i])

['<s>', 'given', 'the', 'large', 'amounts', 'of', 'online', 'textual', 'documents', 'available', 'these', 'days', 'e', 'g', 'news', 'articles', 'weblogs', 'and', 'scientific', 'papers', 'effective', 'methods', 'for', 'extracting', 'keyphrases', 'which', 'provide', 'a', 'highlevel', 'topic', 'description', 'of', 'a', 'document', 'are', 'greatly', 'needed', '</s>', '<s>', 'in', 'this', 'paper', 'we', 'propose', 'a', 'supervised', 'model', 'for', 'keyphrase', 'extraction', 'from', 'research', 'papers', 'which', 'are', 'embedded', 'in', 'citation', 'networks', '</s>', '<s>', 'to', 'this', 'end', 'we', 'design', 'novel', 'features', 'based', 'on', 'citation', 'network', 'information', 'and', 'use', 'them', 'in', 'conjunction', 'with', 'traditional', 'features', 'for', 'keyphrase', 'extraction', 'to', 'obtain', 'remarkable', 'improvements', 'in', 'performance', 'over', 'strong', 'baselines', '</s>']
['<s>', 'numerous', 'works', 'in', 'statistical', 'machine', 'translation', 'have', 'attempte

In [ ]:

#write data into .csv with 2 cols: source & summary
data = {'text':cores_ngram_tokenized, 'summary':abs_sent}
df = pd.DataFrame(data,columns = ['text','summary'])
df.info()
df.to_csv('/content/drive/My Drive/MA_colab/PG_textRank/textRank.csv', index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16297 entries, 0 to 16296
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     16297 non-null  object
 1   summary  16297 non-null  object
dtypes: object(2)
memory usage: 254.8+ KB


In [ ]:
df.head()

,text,summary
0,"[after, annotating, instances, based, on, simp...","[<s>, we, present, a, novel, approach, to, the..."
1,"[however, ,, these, scores, show, that, having...","[<s>, nearly, all, work, in, unsupervised, gra..."
2,"[as, shown, in, figure, 1, ,, we, construct, a...","[<s>, stubs, on, wikipedia, often, lack, compr..."
3,"[lowrank, tensor, representations, following, ...","[<s>, several, compositional, distributional, ..."
4,"[denero, and, uszkoreit, presented, a, preorde...","[<s>, we, present, an, efficient, incremental,..."


In [ ]:
dm_single_close_quote = u'\u2019' # unicode
dm_double_close_quote = u'\u201d'
END_TOKENS = ['.', '!', '?', '...', "'", "`", '"', dm_single_close_quote, dm_double_close_quote, ")"] # acceptable ways to end a sentence

# We use these to separate the summary sentences in the .bin datafiles
SENTENCE_START = '<s>'
SENTENCE_END = '</s>'
VOCAB_SIZE = 200000
CHUNK_SIZE = 500 # num examples per chunk, for the chunked data

In [ ]:
all_train_urls = ""
all_val_urls = ""
all_test_urls = ""
tokenized_dir = default_path + "tokenized_dir/"
finished_files_dir = default_path + "finished_files" #final ouput
chunks_dir = os.path.join(finished_files_dir, "chunked")

In [ ]:
def tokenized_articles (dataset, path):
    for i, row in dataset.iterrows():
        filename = str(i) + '.tok'
        with open(path + filename ,'w',encoding='utf-8') as fh:
            text = row['text']
            tok = text
            summary = row['summary']
            tok.extend(summary)
            list = tok.copy()
            tok_string = "\n".join(str(x) for x in list)
            fh.write(tok_string)

In [ ]:
tokenized_articles(df,tokenized_dir)

In [ ]:
def chunk_file(set_name):
  in_file = finished_files_dir + '/%s.bin' % set_name
  reader = open(in_file, "rb")
  chunk = 0
  finished = False
  while not finished:
    chunk_fname = os.path.join(chunks_dir, '%s_%03d.bin' % (set_name, chunk)) # new chunk
    with open(chunk_fname, 'wb') as writer:
      for _ in range(CHUNK_SIZE):
        len_bytes = reader.read(8)
        if not len_bytes:
          finished = True
          break
        str_len = struct.unpack('q', len_bytes)[0]
        example_str = struct.unpack('%ds' % str_len, reader.read(str_len))[0]
        writer.write(struct.pack('q', str_len))
        writer.write(struct.pack('%ds' % str_len, example_str))
      chunk += 1


def chunk_all():
  # Make a dir to hold the chunks
  if not os.path.isdir(chunks_dir):
    os.mkdir(chunks_dir)
  # Chunk the data
  for set_name in ['train', 'val', 'test']:
    print ("Splitting %s data into chunks..." % set_name)
    chunk_file(set_name)
  print ("Saved chunked data in %s" % chunks_dir)
  

In [ ]:
def get_art_abs(articles,summaries):
    for i in range(len(articles)):
        try:
            article = " ".join(articles[i])
            abstract = " ".join(summaries[i])
        except IndexError:
            print(i)
    return article,abstract

In [ ]:
def write_to_bin(file_names, out_file, makevocab=False):
  """Reads the tokenized .story files corresponding to the urls listed in the url_file and writes them to a out_file."""
  story_fnames = [str(s) + ".tok" for s in file_names]
  #print(story_fnames)
  num_stories = len(story_fnames)
  


  with open(out_file, 'wb') as writer:
      for idx,s in enumerate(story_fnames):
          if idx % 500 == 0:
            print( "Writing story %i of %i; %.2f percent done" % (idx, num_stories, float(idx)*100.0/float(num_stories)))
      

      for i in file_names:
      # Write to tf.Example
          article = " ".join(cores_ngram_tokenized[i])
          abstract = " ".join(abs_sent[i])
        
      
          tf_example = example_pb2.Example()
          tf_example.features.feature['article'].bytes_list.value.extend([article.encode('utf-8')])
          tf_example.features.feature['abstract'].bytes_list.value.extend([abstract.encode('utf-8')])
          tf_example_str = tf_example.SerializeToString()
          #print(tf_example_str)
          str_len = len(tf_example_str)
          writer.write(struct.pack('q', str_len))
          writer.write(struct.pack('%ds' % str_len, tf_example_str))

  print ("Finished writing file %s\n" % out_file)


In [ ]:
def makevocab (train_text,train_ab):
    vocab_counter = collections.Counter()
    for i in range(len(train_text)):
        abs_tokens = [t.lower() for t in train_ab[i] if t not in [SENTENCE_START, SENTENCE_END]]
        train_tokens = [t.lower() for t in train_text[i] if t not in [SENTENCE_START, SENTENCE_END]]
        tokens = train_tokens + abs_tokens
        tokens = [t.strip() for t in tokens] # strip
        tokens = [t for t in tokens if t!=""] # remove empty
        vocab_counter.update(tokens)
    return vocab_counter

In [ ]:
num = df.shape[0]
vocab_counter = makevocab(cores_ngram_tokenized[:num-2000],abs_sent[:num-2000])

In [ ]:
print(len(vocab_counter))
vocab_counter['</s>']

139923


0

In [ ]:
print(len(vocab_counter))
print ("Writing vocab file...")
with open(os.path.join(finished_files_dir, "vocab"), 'w', encoding="utf-8") as writer:
      for word, count in vocab_counter.most_common(VOCAB_SIZE):
        writer.write(word + ' ' + str(count) + '\n')
print ("Finished writing vocab file")


139923
Writing vocab file...
Finished writing vocab file


In [ ]:
if __name__ == '__main__':

  num = df.shape[0]


  all_train_urls = range(0,num-2000)
  all_val_urls = range(num-2000,num-1000)
  all_test_urls = range(num-1000,num)

  #for testing
  #all_train_urls= range(0,5)
  #all_val_urls = range(5,10)
  #all_test_urls = range(10,15)

  # Read the tokenized stories, do a little postprocessing then write to bin files
  write_to_bin(all_test_urls, os.path.join(finished_files_dir, "test.bin"))
  write_to_bin(all_val_urls, os.path.join(finished_files_dir, "val.bin"))
  write_to_bin(all_train_urls, os.path.join(finished_files_dir, "train.bin"), makevocab=True)

  # Chunk the data. This splits each of train.bin, val.bin and test.bin into smaller chunks, each containing e.g. 1000 examples, and saves them in finished_files/chunks
  chunk_all()

Writing story 0 of 1000; 0.00 percent done
Writing story 500 of 1000; 50.00 percent done
Finished writing file /content/drive/My Drive/MA_colab/PG_textRank/finished_files/test.bin

Writing story 0 of 1000; 0.00 percent done
Writing story 500 of 1000; 50.00 percent done
Finished writing file /content/drive/My Drive/MA_colab/PG_textRank/finished_files/val.bin

Writing story 0 of 14297; 0.00 percent done
Writing story 500 of 14297; 3.50 percent done
Writing story 1000 of 14297; 6.99 percent done
Writing story 1500 of 14297; 10.49 percent done
Writing story 2000 of 14297; 13.99 percent done
Writing story 2500 of 14297; 17.49 percent done
Writing story 3000 of 14297; 20.98 percent done
Writing story 3500 of 14297; 24.48 percent done
Writing story 4000 of 14297; 27.98 percent done
Writing story 4500 of 14297; 31.48 percent done
Writing story 5000 of 14297; 34.97 percent done
Writing story 5500 of 14297; 38.47 percent done
Writing story 6000 of 14297; 41.97 percent done
Writing story 6500 of 